In [12]:
import pandas as pd
# Load the CSV file
df = pd.read_csv("data/financials_raw.csv", encoding="ISO-8859-1")

/var/folders/wk/s65gzdnd2zq9zgz4swg1smsw0000gn/T/ipykernel_3278/3802505117.py:3: DtypeWarning: Columns (3,4,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/financials_raw.csv", encoding="ISO-8859-1")


In [13]:
# Display the column names
print(df.columns)

Index(['ENTITY_CURRENCY', 'CURRENCY', 'EXCHANGE_RATE', 'SOLD_TO_CITY',
       'SOLD_TO_COUNTRY', 'INVOICE_DATE', 'INVOICE_YEAR', 'INVOICE_MONTH',
       'UNIT_OF_MEASURE', 'INV_LINE_SEQ__CRM_USAGE_ONLY_', 'ITEM_CODE',
       'ITEM_EU_FAMILY', 'IS_INTERCO_FLAG', 'INVOICED_QUANTITY',
       'INVOICE_LIST_PRICE', 'INVOICE_PRICE', 'INVOICE_STD_COST',
       'ITEM_MATERIAL_COST', 'ITEM_LABOR_COST', 'ITEM_BURDEN_COST',
       'ITEM_OVERHEAD_COST', 'ITEM_SUBCONTRACTING_COST', 'ITEM_STANDARD_COST',
       'SALES_FUNCTIONAL_CURRENCY', 'COGS', 'GROSS_AMOUNT', 'DISCOUNT_AMOUNT',
       'FY', 'Financial Period', 'FY Period', 'EntityID', 'InvoiceID',
       'SoldToID', 'ItemCodeID', 'Multiplicator'],
      dtype='object')


In [14]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6439207 entries, 0 to 6439206
Data columns (total 35 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   ENTITY_CURRENCY                object 
 1   CURRENCY                       object 
 2   EXCHANGE_RATE                  float64
 3   SOLD_TO_CITY                   object 
 4   SOLD_TO_COUNTRY                object 
 5   INVOICE_DATE                   object 
 6   INVOICE_YEAR                   int64  
 7   INVOICE_MONTH                  int64  
 8   UNIT_OF_MEASURE                object 
 9   INV_LINE_SEQ__CRM_USAGE_ONLY_  float64
 10  ITEM_CODE                      object 
 11  ITEM_EU_FAMILY                 float64
 12  IS_INTERCO_FLAG                object 
 13  INVOICED_QUANTITY              float64
 14  INVOICE_LIST_PRICE             float64
 15  INVOICE_PRICE                  float64
 16  INVOICE_STD_COST               float64
 17  ITEM_MATERIAL_COST             float64
 18  IT

In [15]:
# Total GROSS_AMOUNT by INVOICE_MONTH AND INVOICE_YEAR, as int, formatted as $#,###,###.##
# Also the count of invoices for each month and year
total_gross_amount = (
    df.groupby(["INVOICE_MONTH", "INVOICE_YEAR"])
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
total_gross_amount['GROSS_AMOUNT'] = total_gross_amount['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
total_gross_amount['INVOICE_COUNT'] = total_gross_amount['INVOICE_COUNT'].astype(int)

# sort by INVOICE_YEAR and INVOICE_MONTH
total_gross_amount = total_gross_amount.sort_values(by=["INVOICE_YEAR", "INVOICE_MONTH"])
print(total_gross_amount)

    INVOICE_MONTH  INVOICE_YEAR     GROSS_AMOUNT  INVOICE_COUNT
26             11          2020       $12,585.31           2042
29             12          2020  $213,571,032.10        1769806
0               1          2021  $174,887,648.00        1370200
3               2          2021  $183,058,237.85        1055032
6               3          2021  $218,297,981.31        1132935
9               4          2021  $218,547,705.31        1013208
12              5          2021  $187,152,210.72         919287
15              6          2021       $41,608.57             21
30             12          2021  $193,484,742.68         880759
1               1          2022  $215,569,187.28         834138
4               2          2022  $204,574,156.10         828166
7               3          2022  $205,778,009.24        1003299
10              4          2022  $190,809,688.94         752260
13              5          2022  $180,731,558.45         907737
16              6          2022  $191,22

In [16]:
# Filter by sales in SPAIN
spain_sales = df[df["SOLD_TO_COUNTRY"] == "SPAIN"]

# Total GROSS_AMOUNT by INVOICE_MONTH AND INVOICE_YEAR, as int, formatted as $#,###,###.##
# Also the count of invoices for each month and year
spain_total_gross_amount = (
    spain_sales.groupby(["INVOICE_MONTH", "INVOICE_YEAR"])
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
spain_total_gross_amount['GROSS_AMOUNT'] = spain_total_gross_amount['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
spain_total_gross_amount['INVOICE_COUNT'] = spain_total_gross_amount['INVOICE_COUNT'].astype(int)

# sort by INVOICE_YEAR and INVOICE_MONTH
spain_total_gross_amount = spain_total_gross_amount.sort_values(by=["INVOICE_YEAR", "INVOICE_MONTH"])
print(spain_total_gross_amount)

    INVOICE_MONTH  INVOICE_YEAR   GROSS_AMOUNT  INVOICE_COUNT
28             12          2020  $3,236,168.04          14932
0               1          2021  $4,145,849.44          10283
3               2          2021  $3,600,682.67          12931
6               3          2021  $3,704,253.79          16454
9               4          2021  $6,413,732.35          23237
12              5          2021  $5,016,890.23          14501
15              6          2021        $100.96              2
29             12          2021  $6,537,842.84          19488
1               1          2022  $6,487,902.66          16394
4               2          2022  $3,966,055.53          16294
7               3          2022  $7,056,941.51          16749
10              4          2022  $3,880,280.38          15006
13              5          2022  $4,053,337.12          17046
16              6          2022  $5,539,376.05          12536
18              7          2022  $3,451,685.63          12652
20      

In [17]:
# Get the breakdown of sales by SOLD_TO_CITY, by ITEM_CODE, by ITEM_EU_FAMILY and by SoldToID in SPAIN for INVOICE_YEAR 2023 INVOICE_MONTH 11
# each in a separate DataFrame, sorted by GROSS_AMOUNT in descending order
spain_sales_2023_11 = spain_sales[(spain_sales["INVOICE_YEAR"] == 2023) & (spain_sales["INVOICE_MONTH"] == 11)]
spain_sales_by_city = (
    spain_sales_2023_11.groupby("SOLD_TO_CITY")
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
spain_sales_by_city = spain_sales_by_city.sort_values(
    by="GROSS_AMOUNT", ascending=False
)
spain_sales_by_city['GROSS_AMOUNT'] = spain_sales_by_city['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
spain_sales_by_city['INVOICE_COUNT'] = spain_sales_by_city['INVOICE_COUNT'].astype(int)
print(spain_sales_by_city)

spain_sales_by_item = (
    spain_sales_2023_11.groupby("ITEM_CODE")
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
spain_sales_by_item = spain_sales_by_item.sort_values(
    by="GROSS_AMOUNT", ascending=False
)
spain_sales_by_item['GROSS_AMOUNT'] = spain_sales_by_item['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
spain_sales_by_item['INVOICE_COUNT'] = spain_sales_by_item['INVOICE_COUNT'].astype(int)
print(spain_sales_by_item)

spain_sales_by_item_family = (
    spain_sales_2023_11.groupby("ITEM_EU_FAMILY")
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
spain_sales_by_item_family = spain_sales_by_item_family.sort_values(
    by="GROSS_AMOUNT", ascending=False
)
spain_sales_by_item_family['GROSS_AMOUNT'] = spain_sales_by_item_family['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
spain_sales_by_item_family['INVOICE_COUNT'] = spain_sales_by_item_family['INVOICE_COUNT'].astype(int)
print(spain_sales_by_item_family)

spain_sales_by_sold_to_id = (
    spain_sales_2023_11.groupby("SoldToID")
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
spain_sales_by_sold_to_id = spain_sales_by_sold_to_id.sort_values(
    by="GROSS_AMOUNT", ascending=False
)
spain_sales_by_sold_to_id['GROSS_AMOUNT'] = spain_sales_by_sold_to_id['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
spain_sales_by_sold_to_id['INVOICE_COUNT'] = spain_sales_by_sold_to_id['INVOICE_COUNT'].astype(int)
print(spain_sales_by_sold_to_id)

             SOLD_TO_CITY   GROSS_AMOUNT  INVOICE_COUNT
219              VALENCIA  $2,167,665.75           4096
115                MADRID    $275,592.07            590
31              BARCELONA    $169,765.27            310
153              PAMPLONA    $106,888.10            213
101   LAS ROZAS DE MADRID     $94,361.52            106
..                    ...            ...            ...
15   ALHAURIN DE LA TORRE          $0.00              4
129              MENGIBAR          $0.00              6
223        VENTA DE BANOS          $0.00              1
70          ESPARRAGALEJO         $-2.53              2
145        OLESA DE MONT.        $-38.28              1

[244 rows x 3 columns]
         ITEM_CODE   GROSS_AMOUNT  INVOICE_COUNT
579  COMMI-ES1-BTB  $1,176,778.68              2
37            1160    $183,795.49             12
36            1159    $131,418.22             21
602      DV1672720    $101,030.00             85
438        1656220     $89,873.42             69
..        